In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [30]:
url = 'https://en.wikipedia.org/wiki/List_of_companies_of_Nepal'
page = requests.get(url)
# print(page.text)
soup = BeautifulSoup(page.content,'html.parser')

In [31]:
table = soup.find('table', class_ = 'wikitable')
columns = table.find_all('th')

headers = [column_name.text.strip() for column_name in columns]
headers

['Name', 'Industry', 'Sector', 'Headquarters', 'Founded', 'Notes']

In [51]:
df = pd.DataFrame(columns = headers)
df

,Name,Industry,Sector,Headquarters,Founded,Notes


In [56]:
row_data = table.find_all('tr')[1:]

for tr in row_data:
    td = tr.find_all('td')
    individual_row_data = [data.text.strip() for data in td]
    df.loc[len(df)] = individual_row_data

In [80]:
# df.to_csv('company_list_Nepal.csv' , index=False)
from sqlalchemy import create_engine,text
import pandas as pd
import logging

# Enable SQLAlchemy logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)


db_username = 'your_username'
db_password = ''
db_host = 'localhost'
db_port = '5432'
db_name = 'your_database_name'

# Create the connection string
connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
# Create the database engine
engine = create_engine(connection_string)

# Read CSV file into a DataFrame
df = pd.read_csv('company_list_Nepal.csv')
# df.head()
# Test the connection (optional)
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        for row in result:
            print(row)
except Exception as e:
    print("Error message:", e)

table_name = 'nepal_company_list'
# Check if the table exists in the database
try:
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT * FROM information_schema.tables WHERE table_name = '{table_name}';"))
        if result.rowcount == 0:
            print("Table does not exist.")
        else:
            print("Table exists.")
except Exception as e:
    print("Error message:", e)


# Load the DataFrame into the PostgreSQL table
try:
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Data loaded successfully into table '{table_name}'")
except Exception as e:
    print("Error while loading data into PostgreSQL:", e)

# Check the data in the table
try:
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT * FROM {table_name} LIMIT 5;"))
        rows = result.fetchall()
        if len(rows) == 0:
            print("Table is empty.")
        else:
            for row in rows:
                print(row)
except Exception as e:
    print("Error message:", e)

INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:select current_schema()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:show standard_conforming_strings
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT version();
INFO:sqlalchemy.engine.Engine:[generated in 0.00093s] {}
INFO:sqlalchemy.engine.Engine:ROLLBACK
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT * FROM information_schema.tables WHERE table_name = 'nepal_company_list';
INFO:sqlalchemy.engine.Engine:[generated in 0.00165s] {}
INFO:sqlalchemy.engine.Engine:ROLLBACK
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.

('PostgreSQL 14.11 (Homebrew) on aarch64-apple-darwin23.2.0, compiled by Apple clang version 15.0.0 (clang-1500.1.0.2.5), 64-bit',)
Table does not exist.
Data loaded successfully into table 'nepal_company_list'
('Agni Air', 'Consumer services', 'Airlines', 'Kathmandu', '2006', 'Airline, defunct 2006')
('Agriculture Development Bank', 'Financials', 'Banks', 'Kathmandu', '1968', 'Bank, rural credit institution')
('Air Nepal International', 'Consumer services', 'Airlines', 'Kathmandu', '2005', 'Airline, defunct 2006')
('Asian Airlines', 'Consumer services', 'Airlines', 'Kathmandu', '2002[6]', 'Airline, defunct 2006')
('Bajeko Sekuwa', 'Consumer services', 'Restaurants & bars', 'Kathmandu', '?', 'Restaurant chain')
